In [1]:
from electra_model_tf2 import TFElectraGen, TFElectraDis
from tokenizers.implementations import SentencePieceBPETokenizer
from CocCocTokenizer import PyTokenizer
import tensorflow as tf
T = PyTokenizer(load_nontone_data=True)

## Tokenizer

In [2]:
tokenizer = SentencePieceBPETokenizer(
    "./vocab/vocab.json",
    "./vocab/merges.txt",
)
tokenizer.add_special_tokens(["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"])

5

In [3]:
def clean_text(txt_raw):
    return ' '.join(T.word_tokenize(txt_raw, tokenize_option=0))

In [4]:
def encode_text(text):
    text_encode = tokenizer.encode(text)
    plain_text = ['[CLS]'] + text_encode.tokens + ['[SEP]']
    indices = [tokenizer.token_to_id("[CLS]")] + text_encode.ids + [tokenizer.token_to_id("[SEP]")]
    assert len(plain_text) == len(indices)
    return plain_text, indices

## Load models

In [5]:
gen_model = TFElectraGen.from_pretrained("./model_pretrained/gen/")

In [6]:
dis_model = TFElectraDis.from_pretrained("./model_pretrained/dis/")

## Test models

In [7]:
text = "trân trọng gửi đến quý thành viên những thông tin pháp luật nổi bật sau đây"
txt_plain, txt_indices = encode_text(clean_text(text))
print(" ".join(txt_plain))

[CLS] ▁trân_trọng ▁gửi ▁đến ▁quý ▁thành_viên ▁những ▁thông_tin ▁pháp_luật ▁nổi_bật ▁sau ▁đây [SEP]


#### Mask an item
mask item at index 2. (word "▁gửi")

In [8]:
masked_index = 2
txt_plain[masked_index] = '[MASK]'
txt_indices[masked_index] = tokenizer.token_to_id('[MASK]')
print(" ".join(txt_plain))
print(txt_indices)

[CLS] ▁trân_trọng [MASK] ▁đến ▁quý ▁thành_viên ▁những ▁thông_tin ▁pháp_luật ▁nổi_bật ▁sau ▁đây [SEP]
[64002, 8806, 64004, 1168, 2673, 2545, 1125, 1705, 2211, 4973, 1288, 1433, 64003]


##### Find masked item

In [9]:
gen_model_output = gen_model.get_output_generator_task(tf.constant([txt_indices]))
gen_model_output.shape

TensorShape([1, 13, 64005])

Find top 50 words for the masked index

In [10]:
print([tokenizer.id_to_token(item) for item in tf.math.top_k(gen_model_output[0, masked_index], 50).indices.numpy().tolist()])

['▁quan_tâm', '▁sâu_sắc', '▁nhớ', '▁nhắc', '▁gửi', '▁tính', '▁cho', '▁liên_quan', '▁đề_cập', '▁mang', '▁tưởng_nhớ', '▁hướng', '▁biết', '▁thông_tin', '▁nhấn_mạnh', '▁chú_ý', '▁tốt_đẹp', '▁trân_trọng', '▁dành', '▁chân_thành', '▁cảm_ơn', '▁kỷ_niệm', '▁điện_mừng', '▁nói', '▁có', '▁đem', '▁giới_thiệu', '▁tin', '▁hơn', '▁gửi_gắm', '▁thông_điệp', '▁lời_chúc', '▁ý_kiến', '▁cảm_nhận', '▁ghi_nhận', '▁sự', '▁dẫn', '▁hình_ảnh', '▁nhận_thức', '▁sâu_đậm', '▁biết_ơn', '▁người', '▁tiếp', '▁đóng_góp', '▁đến', '▁thiết_thực', '▁,', '▁với', '▁nghĩ', '▁rõ']


#### Unmark item
Choose a word and fill to mask index (word '▁quan_tâm' for example)

In [11]:
mark_token = '▁quan_tâm'
txt_plain[masked_index] = mark_token
txt_indices[masked_index] = tokenizer.token_to_id(mark_token)
print(" ".join(txt_plain))
print(txt_indices)

[CLS] ▁trân_trọng ▁quan_tâm ▁đến ▁quý ▁thành_viên ▁những ▁thông_tin ▁pháp_luật ▁nổi_bật ▁sau ▁đây [SEP]
[64002, 8806, 2768, 1168, 2673, 2545, 1125, 1705, 2211, 4973, 1288, 1433, 64003]


#### Find replaced item

In [12]:
dis_model_output = dis_model.get_output_discriminator_task(tf.constant([txt_indices]))
print(dis_model_output.shape)

(13,)


- True mean replaced item
- False mean origin item

In [13]:
plain_result = list(tf.cast(((tf.math.sign(dis_model_output) + 1) / 2), tf.bool).numpy())
print(plain_result)
print("Replaced words: {}".format([txt_plain[i] for i, value in enumerate(plain_result) if value]))

[False, False, True, False, False, False, False, False, False, False, False, False, False]
Replaced words: ['▁quan_tâm']


## Discriminator model extract features

In [14]:
text = "trân trọng gửi đến quý thành viên những thông tin pháp luật nổi bật sau đây"
txt_plain, txt_indices = encode_text(clean_text(text))
print(txt_plain) 
print(txt_indices)

['[CLS]', '▁trân_trọng', '▁gửi', '▁đến', '▁quý', '▁thành_viên', '▁những', '▁thông_tin', '▁pháp_luật', '▁nổi_bật', '▁sau', '▁đây', '[SEP]']
[64002, 8806, 2320, 1168, 2673, 2545, 1125, 1705, 2211, 4973, 1288, 1433, 64003]


In [15]:
dis_model_output = dis_model(tf.constant([txt_indices]))

Discriminator output to detect replaced item

In [16]:
dis_model_output[0].shape

TensorShape([13])

Discriminator output features (12 layers + 1 output layer)

In [17]:
len(dis_model_output[1])

13

In [18]:
dis_model_output[1][-1].shape

TensorShape([1, 13, 256])